<p style="font-size: 35px; color: Orange; text-align: center;">HELIX</p>

In [1]:
import nbformat

import numpy as np
import pandas as pd 
import requests
import time

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By


from bs4 import BeautifulSoup


#Google sheed API
import gspread
from google.oauth2.service_account import Credentials
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

#Execute shell commands
import subprocess

---
---

## Functions

In [2]:
def petition_html(petition):
    #Open the petition
    driver.execute_script(f"window.open('https://jira.globaldevtools.bbva.com/browse/{petition}')")
    tabs = driver.window_handles
    driver.switch_to.window(tabs[-1])

    description = driver.find_elements(By.XPATH, "//div[@class='user-content-block']")
    descriptions = []
    for desc in description:
        descriptions.append(desc.get_attribute('innerHTML'))
    
    
    driver.switch_to.window(tabs[0])
    time.sleep(2)
    return descriptions

---

### Google sheets connection

In [3]:
# Email service account that need to share the google sheet
service_account_email = "matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com"

#email that need to share the google sheet = matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com
scope = [
    'https://spreadsheets.google.com/feeds', 
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
    ]

credentials = "credentials.json"

try: 
    #Cargar credenciales
    creds = ServiceAccountCredentials.from_json_keyfile_name(credentials, scope)

    #Autorizar gspread con las credenciales
    client = gspread.authorize(creds)

    # ID de la hoja de cálculo 
    srpeadsheet_id = "1HHbgp6LdxBA35JJC96eX9ezoxuXiZkJYyZAt2uN1rmk"

    # Open the Google Sheet file by ID
    spreadsheet = client.open_by_key(srpeadsheet_id)

    # List all sheets file
    sheets = spreadsheet.worksheets()
    print(f"Available sheeds: {[sheet.title for sheet in sheets]}")

    # Seleccionar la hoja específica por nombre
    sheet_name = "Octubre"  # Reemplaza con el nombre de tu hoja
    sheet = spreadsheet.worksheet(sheet_name)
    # Obtener los encabezados esperados
    expected_headers = ["Fecha de alta", "Fecha incurrida", "Fecha de fin", "UUAAA", "Código", "SDATOOL", "Feature", "Petición", "Geografía", "Gestor BBDD", "Ámbito", "Responsable", "Validada", "Horas", "Comentarios"]

    # Obtener todos los registros de la hoja
    data = sheet.get_all_records(expected_headers=expected_headers)
    df = pd.DataFrame(data)

    # Mostrar las primeras filas del DataFrame
    print(df.head())

except gspread.exceptions.APIError as e:
    print(f"APIError: {e}")

except Exception as e:
    print(f"Error: {e}")

Error: ('invalid_grant: Invalid JWT Signature.', {'error': 'invalid_grant', 'error_description': 'Invalid JWT Signature.'})


<p style="font-size: 25px; color: #abebc6;">Limpiamos el Data Frame proveniente del 'excel'</p>

In [4]:
# Eliminar filas vacías y columnas que no van
df = df[df["Fecha de alta"] != ""]
try: 
    df.drop(columns=[""], inplace=True)
except KeyError:
    pass
# Convertir las fechas a formato datetime
df[["Fecha de alta", "Fecha incurrida", "Fecha de fin"]] = df[["Fecha de alta", "Fecha incurrida", "Fecha de fin"]].apply(pd.to_datetime, format="%d/%m/%Y")

# Reemplazar celdas vacías por None
df.replace("", None, inplace=True)

df

NameError: name 'df' is not defined

---

In [5]:
import socket

def check_port_in_use(port, host='127.0.0.1'):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        return s.connect_ex((host, port)) == 0

port = 9222
host = '127.0.0.1'
if check_port_in_use(port, host):
    print(f"El puerto {port} está en uso en {host}.")
else:
    print(f"El puerto {port} no está en uso en {host}.")

El puerto 9222 no está en uso en 127.0.0.1.


---

<p style="text-align: center; font-size: 35px; color: #9EDF9C">Operative system</p>

In [6]:
import platform

if platform.system() == "Darwin": # macOS
    # Comand to open chrome in debug mode
    chrome_command = [
        "/Applications/Google Chrome.app/Contents/MacOS/Google Chrome",
        "--remote-debugging-port=9222",
        "--user-data-dir=/Users/matiasblaquier/chrome_debug"
    ]

    # Execute the command
    subprocess.Popen(chrome_command)

    #Configuracion controlador de Safari
    options = webdriver.ChromeOptions()
    options.add_experimental_option('debuggerAddress', 'localhost:9222')

    #configuracion del controladore de Chrome
    driver = webdriver.Chrome(options=options)

elif platform.system() == "Windows":
    from webdriver_manager.chrome import ChromeDriverManager
    
    #Ejecutar el comando en el cmd
    command = '"C:\Program Files\Google\Chrome\Application\chrome.exe" --remote-debugging-port=9222 --user-data-dir="C:\chrome_debug"'
    process = subprocess.Popen(command, shell=True)
    time.sleep(2)

    #Verifield if the port 9222 is open
    command_check = "netstat -ano | findstr 9222"
    process_check = subprocess.Popen(command_check, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process_check.communicate()
    if stdout:
        print("El puerto 9222 está en uso:")
        print(stdout.decode())
    else:
        print("El puerto 9222 no está en uso o hubo un error:")
        print(stderr.decode())

    #Configurar las opciones de Chrome para conectarse a la instancia Chrome existente
    chrome_options = Options()
    chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")

    #Configuración del controlador de Chrome
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)

    html = driver.page_source

<>:25: SyntaxWarning: invalid escape sequence '\P'
<>:25: SyntaxWarning: invalid escape sequence '\P'
C:\Users\matia\AppData\Local\Temp\ipykernel_27728\1609307265.py:25: SyntaxWarning: invalid escape sequence '\P'
  command = '"C:\Program Files\Google\Chrome\Application\chrome.exe" --remote-debugging-port=9222 --user-data-dir="C:\chrome_debug"'


El puerto 9222 está en uso:
  TCP    127.0.0.1:9222         0.0.0.0:0              LISTENING       6148
  TCP    127.0.0.1:9222         127.0.0.1:52246        TIME_WAIT       0
  TCP    127.0.0.1:9222         127.0.0.1:52247        TIME_WAIT       0
  TCP    127.0.0.1:9222         127.0.0.1:52264        FIN_WAIT_2      5648
  TCP    127.0.0.1:9222         127.0.0.1:52265        TIME_WAIT       0
  TCP    127.0.0.1:52264        127.0.0.1:9222         CLOSE_WAIT      9708



---

<p style="text-align: center; font-size: 35px; color: skyblue;">Going into HELIX</p>

### Getting HELIX info

In [8]:
driver.get("https://itsmhelixbbva-smartit.onbmc.com/smartit/app/#/ticket-console")
time.sleep(5)

#Get from HTML WEB
pag_elements = driver.find_elements(By.XPATH, "//div[@class='ngCellText ng-scope col2 colt2']")
content_important = []
for element in pag_elements:
    pag_element = element.get_attribute('innerHTML')
    content_important.append(pag_element[122: 137])

print(len(content_important))
for i in content_important: 
    print(i)

9
WO0000101357523
WO0000101354384
WO0000101350936
WO0000101350890
WO0000101348011
WO0000101340947
WO0000101337874
TAS000020209091
TAS000020208456


---

In [ ]:
df

,Fecha de alta,Fecha incurrida,Fecha de fin,UUAAA,Código,SDATOOL,Feature,Petición,Geografía,Gestor BBDD,Ámbito,Responsable,Validada,Horas,Comentarios,Estado
0,2024-09-24,2024-10-01,2024-10-01,KESJ,DQ-DP-007019,45403,AUDLEGGES4-249,REQ000101515343,España,PosgreSQL,Legacy,David,Si,3,"Pendiente DROP, volver a hacer check-in",Cerrado
1,2024-09-24,2024-10-01,2024-10-01,KESJ,DQ-DP-007019,45403,AUDLEGGES4-249,REQ000101515343,España,PosgreSQL,Legacy,Matías,No,3,"Pendiente DROP, volver a hacer check-in",Cerrado
2,2024-09-27,2024-10-01,2024-10-01,FACC,DQ-DP-007037,34644,ADH-678,OTRSSD-17407,España,Oracle Physics,NextGEN,David,Si,0.5,None,Cerrado
3,2024-09-27,2024-10-01,2024-10-01,KHFC,None,None,None,OTRSSD-17408,None,Oracle Physics,NextGEN,Pedro,No,1,Descartada,Cerrado
4,2024-09-30,2024-10-01,2024-10-01,MCRR,DQ-DP-007041,38358,ADH-690,OTRSSD-17410,Global,Oracle Physics,NextGEN,David,Si,1.5,None,Cerrado
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,2024-11-11,NaT,NaT,KDHL,90,None,None,OTRSSD-18072,España,Oracle Physics,NextGEN,None,None,25/11,Pedir información,Backlog
244,2024-11-12,NaT,NaT,KENA,91,None,None,KENA_20241106115046,España,DB2 Host,HOST,None,None,25/11,Pedir información. Campo nuevo,Backlog
245,2024-11-12,NaT,NaT,KCOG,None,27522,ALPHATC-685,OTRSSD-18089,Global,Oracle Physics,NextGEN,None,None,25/11,None,Backlog
246,2024-11-13,NaT,NaT,KMET,93,44829,WSONEPLA14-879,OTRSSD-18102,Global,Oracle Physics,NextGEN,None,None,25/11,None,Backlog


In [ ]:
df["Ámbito"].groupby(df["Ámbito"]).count()

Ámbito
HOST        35
Legacy      25
NextGEN    188
Name: Ámbito, dtype: int64

In [ ]:
print(df["Fecha de alta"].count(), df["Fecha de fin"].count(), "\n Peticiones pendientes:", df["Fecha de alta"].count() - df["Fecha de fin"].count())

248 185 
 Peticiones pendientes: 63


Attempting to use a delegate that only supports static-sized tensors with a graph that has dynamic-sized tensors (tensor#141 is a dynamic-sized tensor).
[8702:14083:1209/204129.976782:ERROR:ssl_client_socket_impl.cc(878)] handshake failed; returned -1, SSL error code 1, net_error -101
[8702:14083:1209/204129.977166:ERROR:ssl_client_socket_impl.cc(878)] handshake failed; returned -1, SSL error code 1, net_error -101
